In [156]:
# Sinopec
import urllib,hashlib,requests,xmltodict
from pprint import pprint
from bs4 import BeautifulSoup
#test
url = 'http://114.247.40.65:18030/game/gameEsalesServlet.do'
merchant_id = 'agenttest2'
key = '123456789'
merchant_key = key

# # pro
# url = 'http://oil.19ego.cn/game/gameEsalesServlet.do'
# merchant_id = 'OE20170414856325'
# merchant_key = 'UaWucEnyksHuQOKvjRyV2sVT1q0aKtCl'

    
def concat_params(params):
    pairs = []
    for key in sorted(params):
        if key == 'sign':
            continue
        val = params[key]
        if isinstance(val,str):
            val = urllib.parse.quote_plus(val)
        pairs.append("{}={}".format(key, val))
    return '&'.join(pairs+['key={}'.format(merchant_key)])
def gen_sig(params):
    params = concat_params(params)

    to_hash = hashlib.md5(bytes(params,'utf-8')).hexdigest()
    sig = hashlib.md5(bytes(merchant_key+to_hash+merchant_key,'utf-8')).hexdigest()
    return sig

In [103]:
# 3.1 可充值商品查询接口
data={
    'commandid':'gamequery',
    'protocolid':'gmcard',
    'merchantid':merchant_id,
    'version':1,
    'sign':'',
    'mark':''
}
data['sign']=gen_sig(data)
pprint(data)
resp=requests.post(url=url,data=data)

resp.encoding='GBK'
soup = BeautifulSoup(resp.text,"html5lib")
# soup.prettify()

{'commandid': 'gamequery',
 'mark': '',
 'merchantid': 'OE20170414856325',
 'protocolid': 'gmcard',
 'sign': '73dbfe9943bd7dd3e1d978e7639c6a58',
 'version': 1}


In [157]:
# 3.7 订单结果查询接口(新)
order_message={
"taskName":"worker.sinopec.order_status_polling",
"queue":"sinopec_tasks",
"args":{"orderId":"jhb002024117895368"}
}
data={
    'commandid':'orderquerynew',
    'protocolid':'normalnew',
    'merchantid':merchant_id,
    'version':1,
    'sign':'',
    'orderid':order_message['args']['orderId'],
    'mark':''
}
data['sign']=gen_sig(data)
resp=requests.post(url=url,data=data)
resp.encoding='GBK'
soup = BeautifulSoup(resp.text,"html5lib")
print(soup)
response=soup.find('response')
if response is None:
    pass
resp_data={}
for line in response.contents:
    if line.name is None:
        continue
    resp_data[line.name]=line.text
pprint(resp_data)

<!--?xml version="1.0" encoding="GBK"?--><html><head></head><body><response><code>0</code><orderid>jhb002024117895368</orderid><jxorderid>GDOA170926175152692037</jxorderid><orderstatus>0</orderstatus><finishmoney>10</finishmoney><fillmoney>10</fillmoney><finishtime>20170926175159</finishtime><errorcode>0000</errorcode><errorinfo>充值成功</errorinfo></response>ad48e76090150b9c5f1866ce789f906a</body></html>
{'code': '0',
 'errorcode': '0000',
 'errorinfo': '充值成功',
 'fillmoney': '10',
 'finishmoney': '10',
 'finishtime': '20170926175159',
 'jxorderid': 'GDOA170926175152692037',
 'orderid': 'jhb002024117895368',
 'orderstatus': '0'}


In [154]:
data={
    "url":"http://192.168.0.6:8012/share/generate_huoche_img",
    "data":{
        "orderId":"bl002339945185431",
        "trips":"T135",
        "date":"2017-11-13",
        "seatType":"硬座",
        "fromCity":"上海南",
        "toCity":"杭州东",
        "departureTime":"06:21",
        "arrivalTime":"08:18",
        "dateStep":"",
        "departureDate":"11月13日 周一",
        "arrivalDate":"11月13日 周一",
        "duration":"历时1小时57分",
        "name":"测试",
        "seat":"02车厢02号"
    }
}
requests.post(data['url'],json=data['data']).json()

{'code': 0,
 'data': {'url': 'https://dl.otosaas.com/share/huoche/2a48041df8467a85885dffcebf9b7528.png'}}